In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import *

In [2]:
spark=SparkSession.builder.master("local[*]").appName('test').getOrCreate()

In [3]:
# df=spark.read.csv("datasets/covid.csv", header=True)
# df.printSchema()

In [4]:
# sch=StructType([StructField('Sno',StringType(),True),
#                  StructField('Date',StringType(),True),
#                 StructField('Time',StringType(),True),
#                 StructField('State/UnionTerritory',StringType(),True),
#                 StructField('ConfirmedIndianNational',StringType(),True),
#                 StructField('ConfirmedForeignNational',StringType(),True),
#                 StructField('Cured',StringType(),True),
#                StructField('Deaths',StringType(),True),
#                StructField('Confirmed',StringType(),True)])
# df=spark.read.schema(sch).csv("datasets/covid.csv",header=True)

In [5]:
# df_date_time=df.withColumn("Date",to_date("Date","yy/mm/dd")) \
#      .withColumn("Time",to_timestamp("Time",'HH:mm:ss'))

In [6]:
# newdf = df_date_time.select(year(df_date_time.Date).alias('year'),month(df_date_time.Date).alias('month'),dayofmonth(df_date_time.Date).alias('day'))
# newdf.show()

In [7]:
# df=spark.read.option("multiline","true").json('datasets/data.json')


In [8]:
# updatedDF = df.withColumn("dataset",explode(col("dataset"))).select("@context","@id","conformsTo","dataset.@type","dataset.accesslevel","dataset.accrualPeriodicity","dataset.bureauCode","dataset.contactPoint.fn","dataset.distribution.@type","describedBy")
# updatedDF.show()

In [9]:
df_emp=spark.read.options(header='True').csv('datasets/employee/emp.csv')
df_dep=spark.read.options(header='True').csv('datasets/employee/dept.csv')

df_emp_dep=df_emp.join(df_dep,df_emp.dep_id==df_dep.dep_id,"inner")
df_emp_dep.show()

+------+--------+---------+----------+----------+-------+------+------+---------+
|emp_id|emp_name| job_name|manager_id| hire_date| salary|dep_id|dep_id| dep_name|
+------+--------+---------+----------+----------+-------+------+------+---------+
| 68319| KAYLING|PRESIDENT|          |1991-11-18|6000.00|  1001|  1001|Developer|
| 68319| KAYLING|PRESIDENT|          |1991-11-18|6000.00|  1001|  1001|    CIVIL|
| 68319| KAYLING|PRESIDENT|          |1991-11-18|6000.00|  1001|  1001|    Admin|
| 68319| KAYLING|PRESIDENT|          |1991-11-18|6000.00|  1001|  1001|       IT|
| 66928|   BLAZE|  MANAGER|     68319|1991-05-01|2750.00|  3001|  3001|Developer|
| 66928|   BLAZE|  MANAGER|     68319|1991-05-01|2750.00|  3001|  3001|    Admin|
| 66928|   BLAZE|  MANAGER|     68319|1991-05-01|2750.00|  3001|  3001|       IT|
| 66928|   BLAZE|  MANAGER|     68319|1991-05-01|2750.00|  3001|  3001|       IT|
| 66928|   BLAZE|  MANAGER|     68319|1991-05-01|2750.00|  3001|  3001|Developer|
| 67832|   CLARE

In [10]:
df_sel=df_emp_dep.select('job_name','salary').groupby('job_name').agg(avg('salary').alias('Salary'))
df_sel.show()

+---------+------------------+
| job_name|            Salary|
+---------+------------------+
|  ANALYST|            3100.0|
| SALESMAN|            1500.0|
|    CLERK|1123.6842105263158|
|  MANAGER| 2766.785714285714|
|PRESIDENT|            6000.0|
+---------+------------------+



In [22]:
df=df_emp_dep.select('emp_name','job_name','salary')
df.show()

+--------+---------+-------+
|emp_name| job_name| salary|
+--------+---------+-------+
| KAYLING|PRESIDENT|6000.00|
| KAYLING|PRESIDENT|6000.00|
| KAYLING|PRESIDENT|6000.00|
| KAYLING|PRESIDENT|6000.00|
|   BLAZE|  MANAGER|2750.00|
|   BLAZE|  MANAGER|2750.00|
|   BLAZE|  MANAGER|2750.00|
|   BLAZE|  MANAGER|2750.00|
|   BLAZE|  MANAGER|2750.00|
|   CLARE|  MANAGER|2550.00|
|   CLARE|  MANAGER|2550.00|
|   CLARE|  MANAGER|2550.00|
|   CLARE|  MANAGER|2550.00|
|   JONAS|  MANAGER|2957.00|
|   JONAS|  MANAGER|2957.00|
|   JONAS|  MANAGER|2957.00|
|   JONAS|  MANAGER|2957.00|
|   JONAS|  MANAGER|2957.00|
| SCARLET|  ANALYST|3100.00|
| SCARLET|  ANALYST|3100.00|
+--------+---------+-------+
only showing top 20 rows



In [53]:
win=Window.partitionBy("job_name").orderBy(col("salary").desc())

In [59]:
row_col=df_emp.withColumn("row_num",row_number().over(win)).filter(col("row_num")==2)
row_col.drop("row_num")
row_col.show()

+------+--------+--------+----------+----------+-------+------+-------+
|emp_id|emp_name|job_name|manager_id| hire_date| salary|dep_id|row_num|
+------+--------+--------+----------+----------+-------+------+-------+
| 69062|   FRANK| ANALYST|     65646|1991-12-03|3100.00|  2001|      2|
| 69324|  MARKER|   CLERK|     67832|1992-01-23|1400.00|  1001|      2|
| 66928|   BLAZE| MANAGER|     68319|1991-05-01|2750.00|  3001|      2|
| 68454|  TUCKER|SALESMAN|     66928|1991-09-08|1600.00|  3001|      2|
+------+--------+--------+----------+----------+-------+------+-------+



In [61]:
rank_col=df_emp.withColumn("rank_num",rank().over(win)).filter(col("rank_num")==3)
rank_col.drop("rank_num")
rank_col.show()

+------+--------+--------+----------+----------+-------+------+--------+
|emp_id|emp_name|job_name|manager_id| hire_date| salary|dep_id|rank_num|
+------+--------+--------+----------+----------+-------+------+--------+
| 68736|  ADNRES|   CLERK|     67858|1997-05-23|1200.00|  2001|       3|
| 67832|   CLARE| MANAGER|     68319|1991-06-09|2550.00|  1001|       3|
| 65271|    WADE|SALESMAN|     66928|1991-02-22|1350.00|  3001|       3|
| 66564|  MADDEN|SALESMAN|     66928|1991-09-28|1350.00|  3001|       3|
+------+--------+--------+----------+----------+-------+------+--------+



In [63]:
denrank_col=df_emp.withColumn("den_num",dense_rank().over(win)).filter(col("den_num")==2)
denrank_col.drop("row_num")
denrank_col.show()

+------+--------+--------+----------+----------+-------+------+-------+
|emp_id|emp_name|job_name|manager_id| hire_date| salary|dep_id|den_num|
+------+--------+--------+----------+----------+-------+------+-------+
| 69324|  MARKER|   CLERK|     67832|1992-01-23|1400.00|  1001|      2|
| 66928|   BLAZE| MANAGER|     68319|1991-05-01|2750.00|  3001|      2|
| 68454|  TUCKER|SALESMAN|     66928|1991-09-08|1600.00|  3001|      2|
+------+--------+--------+----------+----------+-------+------+-------+



In [70]:
lead_col=df_emp.withColumn("lead_num",lead("salary",1,0).over(win))
lead_col.show()

+------+--------+---------+----------+----------+-------+------+--------+
|emp_id|emp_name| job_name|manager_id| hire_date| salary|dep_id|lead_num|
+------+--------+---------+----------+----------+-------+------+--------+
| 67858| SCARLET|  ANALYST|     65646|1997-04-19|3100.00|  2001| 3100.00|
| 69062|   FRANK|  ANALYST|     65646|1991-12-03|3100.00|  2001|       0|
| 63679|SANDRINE|    CLERK|     69062|1990-12-18| 900.00|  2001| 1400.00|
| 69324|  MARKER|    CLERK|     67832|1992-01-23|1400.00|  1001| 1200.00|
| 68736|  ADNRES|    CLERK|     67858|1997-05-23|1200.00|  2001| 1050.00|
| 69000|  JULIUS|    CLERK|     66928|1991-12-03|1050.00|  3001|       0|
| 65646|   JONAS|  MANAGER|     68319|1991-04-02|2957.00|  2001| 2750.00|
| 66928|   BLAZE|  MANAGER|     68319|1991-05-01|2750.00|  3001| 2550.00|
| 67832|   CLARE|  MANAGER|     68319|1991-06-09|2550.00|  1001|       0|
| 68319| KAYLING|PRESIDENT|          |1991-11-18|6000.00|  1001|       0|
| 64989|  ADELYN| SALESMAN|     66928|

In [71]:
lag_col=df_emp.withColumn("lag_num",lag("salary",1,0).over(win))
lag_col.show()

+------+--------+---------+----------+----------+-------+------+-------+
|emp_id|emp_name| job_name|manager_id| hire_date| salary|dep_id|lag_num|
+------+--------+---------+----------+----------+-------+------+-------+
| 67858| SCARLET|  ANALYST|     65646|1997-04-19|3100.00|  2001|      0|
| 69062|   FRANK|  ANALYST|     65646|1991-12-03|3100.00|  2001|3100.00|
| 63679|SANDRINE|    CLERK|     69062|1990-12-18| 900.00|  2001|      0|
| 69324|  MARKER|    CLERK|     67832|1992-01-23|1400.00|  1001| 900.00|
| 68736|  ADNRES|    CLERK|     67858|1997-05-23|1200.00|  2001|1400.00|
| 69000|  JULIUS|    CLERK|     66928|1991-12-03|1050.00|  3001|1200.00|
| 65646|   JONAS|  MANAGER|     68319|1991-04-02|2957.00|  2001|      0|
| 66928|   BLAZE|  MANAGER|     68319|1991-05-01|2750.00|  3001|2957.00|
| 67832|   CLARE|  MANAGER|     68319|1991-06-09|2550.00|  1001|2750.00|
| 68319| KAYLING|PRESIDENT|          |1991-11-18|6000.00|  1001|      0|
| 64989|  ADELYN| SALESMAN|     66928|1991-02-20|17

In [63]:
dataStruct = [(("James","","Smith"),"36636","M",3000), \
      (("Michael","Rose",""),"40288","M",4000), \
      (("Robert","","Williams"),"42114","M",4000), \
      (("Maria","Anne","Jones"),"39192","F",4000), \
      (("Jen","Mary","Brown"),"","F",-1)]

In [ ]:
schemaStruct=StructType([StructField('name',StructType([StructField('FirstName',StringType(),True),
                                                      StructField('MiddleName',StringType(),True),
                                                      StructField('LastName',StringType(),True)
                                                       ])),
                                                        StructField('id',StringType(),True),
                                                        StructField('gender',StringType(),True),
                                                        StructField('salary',IntegerType(),True)])

In [ ]:
df1=spark.createDataFrame(data=dataStruct,schema=schemaStruct)
df1.show(truncate=False)

In [ ]:
df3=df2.withColumn("new_gender",when(df2.Gender=="M","Male")
                                .when(df2.Gender=="F","Female")
                                .when(df2.Gender.isNull(),"")
                                 .otherwise(df2.Gender))
df3.show()

In [78]:
windowspec=Window.partitionBy('job_name').orderBy(col('salary').desc())

df_emp_row=df_emp.withColumn('row_number',row_number().over(windowspec))
df_emp_row.show(truncate=False)

+------+--------+---------+----------+----------+-------+------+----------+
|emp_id|emp_name|job_name |manager_id|hire_date |salary |dep_id|row_number|
+------+--------+---------+----------+----------+-------+------+----------+
|67858 |SCARLET |ANALYST  |65646     |1997-04-19|3100.00|2001  |1         |
|69062 |FRANK   |ANALYST  |65646     |1991-12-03|3100.00|2001  |2         |
|63679 |SANDRINE|CLERK    |69062     |1990-12-18|900.00 |2001  |1         |
|69324 |MARKER  |CLERK    |67832     |1992-01-23|1400.00|1001  |2         |
|68736 |ADNRES  |CLERK    |67858     |1997-05-23|1200.00|2001  |3         |
|69000 |JULIUS  |CLERK    |66928     |1991-12-03|1050.00|3001  |4         |
|65646 |JONAS   |MANAGER  |68319     |1991-04-02|2957.00|2001  |1         |
|66928 |BLAZE   |MANAGER  |68319     |1991-05-01|2750.00|3001  |2         |
|67832 |CLARE   |MANAGER  |68319     |1991-06-09|2550.00|1001  |3         |
|68319 |KAYLING |PRESIDENT|          |1991-11-18|6000.00|1001  |1         |
|64989 |ADEL

In [86]:
dd=df_emp_row.filter("row_number==1").drop("row_number")
dd.show()

+------+--------+---------+----------+----------+-------+------+
|emp_id|emp_name| job_name|manager_id| hire_date| salary|dep_id|
+------+--------+---------+----------+----------+-------+------+
| 67858| SCARLET|  ANALYST|     65646|1997-04-19|3100.00|  2001|
| 63679|SANDRINE|    CLERK|     69062|1990-12-18| 900.00|  2001|
| 65646|   JONAS|  MANAGER|     68319|1991-04-02|2957.00|  2001|
| 68319| KAYLING|PRESIDENT|          |1991-11-18|6000.00|  1001|
| 64989|  ADELYN| SALESMAN|     66928|1991-02-20|1700.00|  3001|
+------+--------+---------+----------+----------+-------+------+



In [66]:
df_emp_rank=df_emp.withColumn('rank',rank().over(windowspec))
df_emp_rank.show(truncate=False)

+------+--------+---------+----------+----------+-------+------+----+
|emp_id|emp_name|job_name |manager_id|hire_date |salary |dep_id|rank|
+------+--------+---------+----------+----------+-------+------+----+
|67858 |SCARLET |ANALYST  |65646     |1997-04-19|3100.00|2001  |1   |
|69062 |FRANK   |ANALYST  |65646     |1991-12-03|3100.00|2001  |1   |
|63679 |SANDRINE|CLERK    |69062     |1990-12-18|900.00 |2001  |1   |
|69324 |MARKER  |CLERK    |67832     |1992-01-23|1400.00|1001  |2   |
|68736 |ADNRES  |CLERK    |67858     |1997-05-23|1200.00|2001  |3   |
|69000 |JULIUS  |CLERK    |66928     |1991-12-03|1050.00|3001  |4   |
|65646 |JONAS   |MANAGER  |68319     |1991-04-02|2957.00|2001  |1   |
|66928 |BLAZE   |MANAGER  |68319     |1991-05-01|2750.00|3001  |2   |
|67832 |CLARE   |MANAGER  |68319     |1991-06-09|2550.00|1001  |3   |
|68319 |KAYLING |PRESIDENT|          |1991-11-18|6000.00|1001  |1   |
|64989 |ADELYN  |SALESMAN |66928     |1991-02-20|1700.00|3001  |1   |
|68454 |TUCKER  |SAL

JupyterRequireError: notebook/js/codecell: Timeout. Library 'notebook/js/codecell' is not loaded.

In [67]:
df_emp_denrank=df_emp.withColumn('denserank',dense_rank().over(windowspec))
df_emp_denrank.show(truncate=False)

+------+--------+---------+----------+----------+-------+------+---------+
|emp_id|emp_name|job_name |manager_id|hire_date |salary |dep_id|denserank|
+------+--------+---------+----------+----------+-------+------+---------+
|67858 |SCARLET |ANALYST  |65646     |1997-04-19|3100.00|2001  |1        |
|69062 |FRANK   |ANALYST  |65646     |1991-12-03|3100.00|2001  |1        |
|63679 |SANDRINE|CLERK    |69062     |1990-12-18|900.00 |2001  |1        |
|69324 |MARKER  |CLERK    |67832     |1992-01-23|1400.00|1001  |2        |
|68736 |ADNRES  |CLERK    |67858     |1997-05-23|1200.00|2001  |3        |
|69000 |JULIUS  |CLERK    |66928     |1991-12-03|1050.00|3001  |4        |
|65646 |JONAS   |MANAGER  |68319     |1991-04-02|2957.00|2001  |1        |
|66928 |BLAZE   |MANAGER  |68319     |1991-05-01|2750.00|3001  |2        |
|67832 |CLARE   |MANAGER  |68319     |1991-06-09|2550.00|1001  |3        |
|68319 |KAYLING |PRESIDENT|          |1991-11-18|6000.00|1001  |1        |
|64989 |ADELYN  |SALESMAN

In [68]:
df_emp_lag=df_emp.withColumn('lag',lag('salary',1,0).over(windowspec))
df_emp_lag.show(truncate=False)

+------+--------+---------+----------+----------+-------+------+-------+
|emp_id|emp_name|job_name |manager_id|hire_date |salary |dep_id|lag    |
+------+--------+---------+----------+----------+-------+------+-------+
|67858 |SCARLET |ANALYST  |65646     |1997-04-19|3100.00|2001  |0      |
|69062 |FRANK   |ANALYST  |65646     |1991-12-03|3100.00|2001  |3100.00|
|63679 |SANDRINE|CLERK    |69062     |1990-12-18|900.00 |2001  |0      |
|69324 |MARKER  |CLERK    |67832     |1992-01-23|1400.00|1001  |900.00 |
|68736 |ADNRES  |CLERK    |67858     |1997-05-23|1200.00|2001  |1400.00|
|69000 |JULIUS  |CLERK    |66928     |1991-12-03|1050.00|3001  |1200.00|
|65646 |JONAS   |MANAGER  |68319     |1991-04-02|2957.00|2001  |0      |
|66928 |BLAZE   |MANAGER  |68319     |1991-05-01|2750.00|3001  |2957.00|
|67832 |CLARE   |MANAGER  |68319     |1991-06-09|2550.00|1001  |2750.00|
|68319 |KAYLING |PRESIDENT|          |1991-11-18|6000.00|1001  |0      |
|64989 |ADELYN  |SALESMAN |66928     |1991-02-20|17

In [69]:
df_emp_lead=df_emp.withColumn('lead',lead('salary',1,0).over(windowspec))
df_emp_lead.show(truncate=False)

+------+--------+---------+----------+----------+-------+------+-------+
|emp_id|emp_name|job_name |manager_id|hire_date |salary |dep_id|lead   |
+------+--------+---------+----------+----------+-------+------+-------+
|67858 |SCARLET |ANALYST  |65646     |1997-04-19|3100.00|2001  |3100.00|
|69062 |FRANK   |ANALYST  |65646     |1991-12-03|3100.00|2001  |0      |
|63679 |SANDRINE|CLERK    |69062     |1990-12-18|900.00 |2001  |1400.00|
|69324 |MARKER  |CLERK    |67832     |1992-01-23|1400.00|1001  |1200.00|
|68736 |ADNRES  |CLERK    |67858     |1997-05-23|1200.00|2001  |1050.00|
|69000 |JULIUS  |CLERK    |66928     |1991-12-03|1050.00|3001  |0      |
|65646 |JONAS   |MANAGER  |68319     |1991-04-02|2957.00|2001  |2750.00|
|66928 |BLAZE   |MANAGER  |68319     |1991-05-01|2750.00|3001  |2550.00|
|67832 |CLARE   |MANAGER  |68319     |1991-06-09|2550.00|1001  |0      |
|68319 |KAYLING |PRESIDENT|          |1991-11-18|6000.00|1001  |0      |
|64989 |ADELYN  |SALESMAN |66928     |1991-02-20|17

In [72]:
df_emp.union(df_dep).show()

AnalysisException: Union can only be performed on tables with the same number of columns, but the first table has 7 columns and the second table has 2 columns;
'Union false, false
:- Relation [emp_id#17,emp_name#18,job_name#19,manager_id#20,hire_date#21,salary#22,dep_id#23] csv
+- Relation [dep_id#48,dep_name#49] csv


In [74]:
df1=df_emp.select('dep_id')
df2=df_dep.select('dep_id')

df1.union(df2).show() #same column name need in union and unionall

+------+
|dep_id|
+------+
|  1001|
|  3001|
|  1001|
|  2001|
|  2001|
|  2001|
|  2001|
|  3001|
|  3001|
|  3001|
|  3001|
|  2001|
|  3001|
|  1001|
|  1001|
|  1001|
|  1001|
|  2001|
|  2001|
|  2001|
+------+
only showing top 20 rows



In [83]:
df_duplicate= df_emp.select("job_name","dep_id","salary").orderBy("job_name",col("dep_id").desc(),col("salary").desc()).drop_duplicates(["dep_id"])
df_duplicate.show()

+--------+------+-------+
|job_name|dep_id| salary|
+--------+------+-------+
|   CLERK|  1001|1400.00|
| ANALYST|  2001|3100.00|
|   CLERK|  3001|1050.00|
+--------+------+-------+



In [99]:
stud=spark.read.options(header='True', inferSchema='True').csv('datasets/student.csv')
stud.show()

+-------+-----------+-----+
|student|    subject|marks|
+-------+-----------+-----+
|  Jacob|Mathematics|  100|
|  Jacob|    Science|   95|
|  Jacob|  Geography|   90|
| Amilee|Mathematics|   90|
| Amilee|    Science|   90|
| Amilee|  Geography|  100|
+-------+-----------+-----+



In [102]:
df_stud=stud.groupBy('student').pivot('subject').max('marks').show()

+-------+---------+-----------+-------+
|student|Geography|Mathematics|Science|
+-------+---------+-----------+-------+
|  Jacob|       90|        100|     95|
| Amilee|      100|         90|     90|
+-------+---------+-----------+-------+

